Determine the transformation of an aperture diameter of 7.5 kpc at the redshift of abell 370 to pixels. First transform the diameter in absolute size (kpc) to apparent size (arcseconds). Then transform apparent size to pixels using WCS.

In [1]:
import numpy as np 

from astropy.io import fits
from astropy import units as u 
from astropy import cosmology
from astropy.wcs import WCS

### Load Data

In [2]:
input_data_path = "https://archive.stsci.edu/pub/hlsp/frontier/abell370/images/hst/v1.0-epoch2/hlsp_frontier_hst_wfc3-30mas-bkgdcor_abell370_f105w_v1.0-epoch2_drz.fits"

hdul = fits.open(input_data_path)
hdu = hdul[0]

data = hdu.data
header = hdu.header
wcs = WCS(header)

### Given

In [3]:
# Given 
aperture_diameter = (7.5 * u.kpc).to(u.Mpc)

NED data for abell 370:

http://ned.ipac.caltech.edu/cgi-bin/objsearch?search_type=Obj_id&objid=132527&objname=1&img_stamp=YES&hconst=73.0&omegam=0.27&omegav=0.73&corr_z=1

In [4]:
"""
Cosmology-Corrected Quantities [Ho =  70.50 km/sec/Mpc, Ωmatter =   0.27, Ωvacuum =   0.73]
[Redshift 0.374247 as corrected to the Reference Frame defined by the 3K Microwave Background Radiation]
Luminosity Distance        :     2011 Mpc      (m-M) = 41.52 mag
Angular-Size Distance      :     1065 Mpc      (m-M) = 40.14 mag
Co-Moving Radial Distance  :     1463 Mpc      (m-M) = 40.83 mag
Co-Moving Tangential Dist. :     1463 Mpc      (m-M) = 40.83 mag
Co-Moving Volume           : 13.1 Gpc^3
Light Travel-Time          :    4.073 Gyr
Age at Redshift 0.374247   :    9.698 Gyr
Age of Universe            :   13.770 Gyr
Scale (Cosmology Corrected):    5161 pc/arcsec =  5.161 kpc/arcsec = 309.67 kpc/arcmin =  18.58 Mpc/degree
Surface Brightness Dimming : Flux Density per Unit Area =  0.28038; Magnitude per Unit Area = 1.381 mag
"""

'\nCosmology-Corrected Quantities [Ho =  70.50 km/sec/Mpc, Ωmatter =   0.27, Ωvacuum =   0.73]\n[Redshift 0.374247 as corrected to the Reference Frame defined by the 3K Microwave Background Radiation]\nLuminosity Distance        :     2011 Mpc      (m-M) = 41.52 mag\nAngular-Size Distance      :     1065 Mpc      (m-M) = 40.14 mag\nCo-Moving Radial Distance  :     1463 Mpc      (m-M) = 40.83 mag\nCo-Moving Tangential Dist. :     1463 Mpc      (m-M) = 40.83 mag\nCo-Moving Volume           : 13.1 Gpc^3\nLight Travel-Time          :    4.073 Gyr\nAge at Redshift 0.374247   :    9.698 Gyr\nAge of Universe            :   13.770 Gyr\nScale (Cosmology Corrected):    5161 pc/arcsec =  5.161 kpc/arcsec = 309.67 kpc/arcmin =  18.58 Mpc/degree\nSurface Brightness Dimming : Flux Density per Unit Area =  0.28038; Magnitude per Unit Area = 1.381 mag\n'

In [5]:
abell370_z = 0.375000
abell370_distance = 1463 * u.Mpc # Comoving

### Equations

\begin{align}
\theta = \frac{(z+1) * Diameter}{Distance_{Comoving}} =  \frac{Diameter}{Distance_{Angular Diameter} } \\
\end{align}

### Compute angular diameter distance

In [6]:
def compute_angular_diameter_distance(d, z):
    return d/(z+1)

In [7]:
angular_diameter_distance = compute_angular_diameter_distance(abell370_distance, abell370_z)
angular_diameter_distance

<Quantity 1064. Mpc>

In [8]:
# check astropy cosmology
cosmology.WMAP5.angular_diameter_distance(abell370_z)

<Quantity 1068.40525789 Mpc>

### Find angular diameter

In [9]:
angular_diameter = np.tan((aperture_diameter / angular_diameter_distance).value) * u.rad 
angular_diameter 

<Quantity 7.04887218e-06 rad>

In [10]:
# check astropy cosmology
(cosmology.WMAP5.arcsec_per_kpc_proper(abell370_z) * aperture_diameter.to('kpc')).to('rad')

<Quantity 7.01980821e-06 rad>

### Convert to arcsec

In [11]:
angular_diameter = angular_diameter.to("arcsec")
angular_diameter

<Quantity 1.45393425 arcsec>

### Find Pixel Size 

In this section we convert the angular size into pixel size. The way we do this is by adding the angular size to the `CRVAL` of the WCS and then converting that world coordinate to pixel values. After getting the new pixel values we subtract the `CRPIX` values to find the difference (angular size). Because RA-DEC is an equatorial coordinate system, I have decided its best to add the angular size to the Dec component of the WCS CRVAL (Dec axis is a great circle). It looks like the image `CRVAL` is sufficiently away from the pols of the celestial sphere, so no need to worry about loops around +/- 90 degrees.  

In [12]:
# Load center pixel from data's WCS 

ra_0 = wcs.wcs.crval[0] * u.Unit(wcs.wcs.cunit[0])
dec_0 = wcs.wcs.crval[1] * u.Unit(wcs.wcs.cunit[1])

print(ra_0, ",",  dec_0)

39.96301 deg , -1.5882933 deg


In [13]:
# Add angular_diameter to center Dec value 

ra_1  = ra_0
dec_1 = dec_0 + angular_diameter

print(ra_1, ",",  dec_1)

39.96301 deg , -1.5878894293737262 deg


In [14]:
# Convert ra_1, dec_1 into pixels

world = np.array([[ra_1.value, dec_1.value],], dtype=np.float64) * u.deg
pixcrd = wcs.wcs_world2pix(world, 1)

pixcrd

array([[4800.        , 6648.46447515]])

In [15]:
# Look at the center pixel values
wcs.wcs.crpix

array([4800., 6600.])

In [16]:
# Find the difference b/w the center pixel and the angular offset pixel:
pixel_diff =  pixcrd - wcs.wcs.crpix 
assert abs(pixel_diff[0,0]) < 1e-10
pixel_diff[0,1]

48.46447515439377

In [17]:
# Find ceiling of pixel diff to conclude pixel size
pixel_size = int(np.ceil(pixel_diff[0,1]))

pixel_size

49